In [1]:
import numpy as np
import copy
from nltk.tokenize import sent_tokenize
import torch
import torch.optim as optim
from torch import nn
import sys
import urllib

if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

In [2]:
# data = urllib.request.urlopen('https://raw.githubusercontent.com/hoanghuy89/rnn-lam-tho-tieng-viet/main/bai_van_mau.txt').read()
# data = data.decode('utf-8')
with open('/data/bai_van_mau.txt', 'r', encoding='utf-8') as f:
    data = f.read()
# data = data.lower()
chars = list(set(data))

char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
vocab_size =  len(chars)

hidden_size = 512
sequence_len = 100
batch_size = 200
len(chars)

FileNotFoundError: [Errno 2] No such file or directory: '/data/bai_van_mau.txt'

In [332]:
# split test to batch_size chunks so we can sample each chunk to build linear batch
def build_data(data_input):
    """
    Create a training set by scanning a window of size sequence_len over the text corpus, with stride 3.
    """
    num_chunk = batch_size
    data_start = int(np.random.choice(range(sequence_len),1))
    data = data_input[data_start:]
    datasize = len(data)
    # Y last chunk last character
    if datasize % num_chunk == 0:
        data += '.'

    chunk_len = int(np.floor(datasize / num_chunk))
    start_index = [chunk_len*i for i in range(num_chunk)]
    chunk_len_cutoff = int(sequence_len*(np.floor(chunk_len / sequence_len))) # fit to sequence length
    

    X = [[] for i in range(num_chunk)]
    Y = [[] for i in range(num_chunk)]

    for i in range(0, chunk_len_cutoff, sequence_len):
        for j in range(num_chunk):
            X[j].append(data[start_index[j]+i: start_index[j]+i + sequence_len])
            Y[j].append(data[start_index[j]+i+1: start_index[j]+i+1 + sequence_len])
    # this need investigation
    X = np.array(X)
    Y = np.array(Y)
    return X, Y
    
X, Y = build_data(data)
X.shape,  len(X[0,0]), "num_chunk, num_sequence, sequence_len",  Y.shape

((200, 987), 100, 'num_chunk, num_sequence, sequence_len', (200, 987))

In [333]:
def vectorization(X, Y):
    """
    Input each sequence (batch_size, sequence_len)
    Convert X and Y (lists or chars) into pytorch tensor to be given to a recurrent neural network.
    """
    batch_size = X.shape[0]
    sequence_len = len(X[0])
    
    x = np.zeros((sequence_len, vocab_size, batch_size))
    y = np.zeros((sequence_len, batch_size))
    for i, chunk in enumerate(X):
        for k, char in enumerate(chunk):
            x[k, char_to_ix[char], i] = 1
    for i, chunk in enumerate(Y):
        for k, char in enumerate(chunk):
            y[k, i] = char_to_ix[char] 
        
    x = torch.LongTensor(x).to(device)
    y = torch.LongTensor(y).to(device)
    return x, y

# test vectorization
x,y = vectorization(X[:,0],Y[:,0])
x.shape,y.shape, 'sequence_len, vocab_size, batch_size'

(torch.Size([100, 244, 200]),
 torch.Size([100, 200]),
 'sequence_len, vocab_size, batch_size')

In [334]:
X[0,:]

array(['a dân tộc.\n- Năm 1925, ông tham gia cách mạng, từng giữ nhiều cương vị quan trọng trong bộ máy lãnh ',
       'đạo của Đảng và Nhà nước Việt Nam.\n- Phạm Văn Đồng từng là Thủ tướng Chính phủ, là học trò và người ',
       'cộng sự gần gũi của Chủ tịch Hồ Chí Minh.\n- Ông có nhiều công trình nghiên cứu, bài nói và bài viết ',
       'về văn hóa, văn nghệ, về Chủ tịch Hồ Chí Minh và danh nhân văn hóa của dân tộc.\n- Những tác phẩm của',
       ' ông thu hút người đọc bằng tư tưởng sâu sắc và giản dị, tình cảm sôi nổi, lời văn trong sáng hấp dẫ',
       'n .\nBài Nguyễn Đình Chiểu, ngôi sao sáng trong văn nghệ của dân tộc của ông được viết nhân kỷ niệm 7',
       '5 năm ngày mất của Nguyễn Đình Chiểu (3 - 7 - 1888)\n- Đặt vấn đề trực tiếp: Nguyễn Đình Chiểu, một n',
       'hà thơ lớn của nước ta, đáng lẽ phải được sáng tỏ hơn nữa trong bầu trời văn nghệ của dân tộc.\n- Hìn',
       'h ảnh ẩn dụ “ngôi sao”, “những vì sao có ánh sáng khác thường”, “con mắt chúng ta phải chăm chú n

In [335]:
def initialize_lstm_parameters():
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    Wf = torch.randn(hidden_size, vocab_size + hidden_size, device=device)*0.01 
    bf = torch.ones((hidden_size, 1), requires_grad=True, device=device)
    Wu = torch.randn(hidden_size, vocab_size + hidden_size, device=device)*0.01 
    bu = torch.zeros((hidden_size, 1), requires_grad=True, device=device)
    Wcc = torch.randn(hidden_size, vocab_size + hidden_size, device=device)*0.01 
    bcc = torch.zeros((hidden_size, 1), requires_grad=True, device=device)
    Wo = torch.randn(hidden_size, vocab_size + hidden_size, device=device)*0.01
    bo = torch.zeros((hidden_size, 1), requires_grad=True, device=device)
    Wy = torch.randn(vocab_size, hidden_size, device=device)*0.01
    by = torch.zeros((vocab_size, 1), requires_grad=True, device=device)
    
    Wf.requires_grad = True
    Wu.requires_grad = True
    Wcc.requires_grad = True
    Wo.requires_grad = True
    Wy.requires_grad = True
    
    
    parameters = [Wf, bf, Wu, bu, Wcc, bcc, Wo, bo, Wy, by]
    
    return parameters

# test init paramter
parameters = initialize_lstm_parameters()
parameters[1].device, parameters[1].is_leaf, parameters[1].requires_grad

(device(type='cuda', index=0), True, True)

In [336]:
def lstm_step_forward(x, a_prev, c_prev, parameters):
    Wf, bf, Wu, bu, Wcc, bcc, Wo, bo, Wy, by = parameters
    
    concat = torch.cat((a_prev, x),axis=0)

    f = torch.sigmoid(Wf @ concat + bf)
    u = torch.sigmoid(Wu @ concat + bu)
    cc = torch.tanh(Wcc @ concat + bcc)
    o = torch.sigmoid(Wo @ concat + bo)


    c = f*c_prev + u*cc
    a = o*torch.tanh(c)
    y = Wy @ a + by
    return y, a, c
#     return a, c

# test forward step
# parameters = initialize_lstm_parameters()
a_prev = torch.randn((hidden_size,batch_size)).to(device)
c_prev = torch.randn((hidden_size,batch_size)).to(device)
x = torch.zeros((vocab_size, batch_size)).to(device)
y, a, c = lstm_step_forward(x, a_prev, c_prev, parameters)
y.shape, a.shape, c.shape

(torch.Size([244, 200]), torch.Size([512, 200]), torch.Size([512, 200]))

In [337]:
def lstm_forward(batch_X, a_prev, c_prev, parameters):
    Wf, bf, Wu, bu, Wcc, bcc, Wo, bo, Wy, by = parameters
    #x = np.zeros((num_sequence, sequence_len, vocab_size, batch_size))
    batch_size = batch_X.shape[-1]
    sequence_len = batch_X.shape[0]
    
    y_hat = torch.zeros((sequence_len, vocab_size, batch_size)).to(device)
    a = torch.zeros((sequence_len+1, hidden_size, batch_size)).to(device)
    c = torch.zeros((sequence_len+1, hidden_size, batch_size)).to(device)
    a[0] = a_prev
    c[0] = c_prev
   
    for t in range(sequence_len):
        y_hat[t], a[t+1], c[t+1] = lstm_step_forward(batch_X[t], a[t].clone(), c[t].clone(), parameters)
    
    # TODO: investigate backward from loss
    # loss = torch.sum(-y*torch.log(y_hat))
    return y_hat, a[-1].detach(), c[-1].detach()

# test forward operation
parameters = initialize_lstm_parameters()
x,y = vectorization(X[:,0],Y[:,0])
a_prev = torch.randn((hidden_size,batch_size)).to(device)
c_prev = torch.randn((hidden_size,batch_size)).to(device)
for i in range(10):
    y_hat, a_prev, c_prev = lstm_forward(x, a_prev, c_prev, parameters)
y_hat.shape, a_prev.shape, c_prev.shape

(torch.Size([100, 244, 200]), torch.Size([512, 200]), torch.Size([512, 200]))

In [358]:
def pickle_compress(file_path, data=None, operation='load'):
    import bz2
    import pickle
    import _pickle as cPickle
    
    a = None
    if operation != 'load':
        with bz2.BZ2File(file_path, 'wb') as f:
            cPickle.dump(data, f)
    else:
        with bz2.BZ2File(file_path, 'rb') as f:
            a = cPickle.load(f)
    return a


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64').reshape(-1)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
   

    return preds
#     return np.argmax(preds)

def softmax1d(x):
    max, _ = torch.max(x, dim=0, keepdims=True) #returns max of each row and keeps same dims
    e_x = torch.exp(x - max) #subtracts each row with its max value
    sum = torch.sum(e_x, dim=0, keepdims=True) #returns sum of each row and keeps same dims
    sf = e_x / sum 

    return sf


def generate_output(parameters, sample_len = 1000, get_input=False):
    '''
    Generate n samples characters with random or set input
    '''
    generated = ''
    num_sequence = X.shape[1]
    a_prev = torch.randn((hidden_size,1)).to(device)
    c_prev = torch.randn((hidden_size,1)).to(device)
    
    if get_input == False:
        idx = int(np.random.choice(range(num_sequence),1))
        sentence = X[0,idx]
        usr_input = sentence
    else:
        usr_input = input("Nhập câu đầu: ")
        sentence = usr_input
        print(sentence)
        
    X_sample = np.array(sentence).reshape(1)
    Y_sample = np.array(sentence[1:]+'').reshape(1)

    generated += usr_input
    x , y = vectorization(X_sample, Y_sample)
    for item in x:
        preds, a_prev, c_prev = lstm_step_forward(item, a_prev, c_prev, parameters)

    for i in range(sample_len):
        preds = softmax1d(preds)
        preds = preds.detach().cpu().numpy().ravel()
#         preds = sample(preds, temperature=.5)
        next_index = np.random.choice(range(vocab_size), p = preds)
        x = torch.zeros((vocab_size,1)).to(device)
        x[next_index,0]=1
        generated += ix_to_char[next_index]
        preds, a_prev, c_prev = lstm_step_forward(x, a_prev, c_prev, parameters)
        
    return generated
# parameters = initialize_lstm_parameters()
generate_output(parameters)

'ng). Giải thích vì sao có sự khác nhau đó.\n- Khi nói phân tích một truyện ngắn là nói đến thao tác của bảo vệ tự học, sống vị tha, cố gắng chạy bọt thực hiện theo để giúp con người và thế giới đoàn kết. Đnợng giảng dạy ngữ mong có thể nộp tự do, phù hợp, coi trọng rõ ràng, có thể gợi nhắn lại được nó cần trầm quả. Đây là nhà thì quyền được chính nghĩa, kém và hiệu quả của hạnh phúc.\nNhững nơi và thì dù nghèo khổ cũng không nhìn thấy vài năm sống. Trao đoan tích cực của chúng ta đã thể hiện mình!\n– Đầu năm 1938, chúng ta cố ginh hoàng êm dịu ở Thanh Thảo. Cô giáo Việt trường rất đơn giản. Nước 12 mùa hè sao mà bố đồng thời. Nhưng rồi, sau mái tóc đen hay mấy ngăn thét và vẫy xuất tất vỗ trái đi bô\'. Nuôi chơi và khen là bà. Mọi thứ nhỏ xinh ấy, phía nhà em vực nhỏ. Nàng bớt mờ ảo màu xanh trái, cây bàng vàng lên và tàn phai mời ông mặt trời như muốn thu nhỏ lại để trong trẻo khác và nắng ấm. Cây đã đẹp (GV chia sẻ): Phong trại, Thạch Sanh các biển thân trong lịch sử, những chinh ph

# Training the network

In [339]:
num_iterations = 1000
num_sequence = X.shape[1]
# if 'parameters' not in locals():
parameters = initialize_lstm_parameters()

optimizer = optim.Adam(parameters, lr=0.01)
criterion = nn.CrossEntropyLoss().to(device)

In [342]:
def train():
    # m = len(X)
#     num_batch = np.floor(m/batch_size)
    # m_roundoff = int(batch_size*num_batch)
    
    a_prev = torch.randn((hidden_size,batch_size)).to(device)
    c_prev = torch.randn((hidden_size,batch_size)).to(device)
    count = 0

    for i in range(num_iterations):
        X, Y = build_data(data) # randomize where to start string sequence
        loss_total = 0
    #         permutation = np.random.permutation(m)
    #         for j in range(0,m_roundoff,batch_size):
    #             indices = permutation[j:j+batch_size]
        for j in range(0,num_sequence):
    #             batch_X, batch_Y = [X[i] for i in indices], [Y[i] for i in indices] # mini batch
    #             batch_X, batch_Y = vectorization(batch_X[:,j], batch_Y[:,j])
            batch_X, batch_Y = vectorization(X[:,j], Y[:,j])
            y_hat, a_prev, c_prev =  lstm_forward(batch_X, a_prev, c_prev, parameters)

    #         loss = criterion(y_hat.T, batch_Y)
            loss = criterion(y_hat.T, batch_Y.T)
            
            loss.backward()
            torch.nn.utils.clip_grad_value_(parameters, 5)
            optimizer.step()    # Does the update
            optimizer.zero_grad()
            count+=1
            if count % 100 == 0:
                print(generate_output(parameters, sample_len=1000))
#             print(loss.detach().cpu())
            loss_total += loss.detach()
            
    #         pickle_compress('params.pickle', parameters, 'write')
        print('\nIteration: %d, Loss: %f' % (i, loss_total/num_sequence) + '\n')
        pickle_compress('params.pickle', data=parameters, operation='write')

#         print(generate_output(parameters, sample_len=1000))
train()

ia đình anh Tràng cũng cảm thấy vui vẻ phấn khởi. Có thể thấy trong nạn đói, tình người vẫn lấp lánh với học cho đội cho người chú thế dù thế biết thức thức như chúng thương bà vào được là người của học là người thiết thì bó dà hơi là và giáo để đinh được hành người chú những chú giường để hiện chủ và thiết người với bàng em rất thế hoa được thơ chiến ngày thành cho chuyện hất thiến thiể  nha như thường thì mà thành thiện thật đi thân thạnh thơ thàng bằng người và là một người trong thức trong ngheo thực trường giời xây có giữ đình trước thần thương của động thơ của sau là một những bà chuy nhà và chuyến cho cá hương bàn thứ như được thác gia như và thương người trên sống rất là mà thì người người chi trường trên một và thơ đến được là đã chi cho người bàn thận đường nghĩa người trương học vào nghĩa được thắc những là hình thương ngày người mài chú chỉ nhưng người cho người như nghĩa vào thường cho chị một thức cần thước chúng đi có rất biết thạng cho như người cho chính thiếng là vào 

nó qua sông tao cũng qua sông, nó về quận tao cũng tới. Một tay tao bồng em mày, một tay tao cắp rổ. Chính chú thỏ thư tha thiết chúng ta trên đồng thời của thành phố như những câu trơ cho con người và con người như thế nào? Thiếu nhiệt vui vẻ trong cuộc sống của trong thiên nhiên khiến cho đến người ta có thể thấy được mặt trời như màu xanh lá cho mọi người thì từ nghệ thuật.
Trưởng thành vì đó là chị hoạt động mạnh mẽ và đẹp đến những chú và mẹ có thể thấy rõ ràng làm cho nhau như thế nào? Những loại người đã giúp đỡ các chứa chúng ta có thể làm thay lên chúng ta có thể nào có thể làm việc thì thiên nhiên và tham quan thì những chiếc vật chúng ta có thể không thể thiết theo nhiều thế hệ thế giới, không phải chỉ không có thể làm được thì sẽ thấy bản thân mình cũng là thức trang nhân vật thể hiện về các thầy cô giáo dục và trở thành những chiếc vì vô cùng là một cánh đồng thầy chú là những thứ văn học của các em thuộc thể hiện luôn chiến chúng của các bạn giảng nhựa và chỉ về câu thơ v

n nhủ cho hai chị em Chiến và Việt: "chuyện gia đình ta nó cũng dài như sông, để rồi chú sẽ chia cho chúng ta đã thấy những chiến tranh thể hiện được tình yêu đất nước đã được nhìn thấy những kỉ niệm văn học thực tại, cho mình những chiếc lá to trông rất đẹp như chiếc cánh cây có màu xanh thẫm thiết tha của mỗi người chú chó nhà em đã hạnh phúc. Chú chó nhà em còn thấy thương hình thời gian để học thật tốt, tương trường thức dậy. Chính vì vậy mà tôi chỉ thấy được thể hiện hai chị em đã thấy được thể hiện được hình ảnh đầu tiên đã văn chương đã trở thành những cây chuối là một nhà mà chúng em đã trở thành một người đọc đã thấy hát những cây hoa hồng chuối của chúng ta còn nhớ những con người nghỉ học mà còn được học thật đẹp trên chiếu trước bây giờ thật đẹp. Đến năm 1981, điểm trung thủ thiên nhiên thuộc và sáng tạo ra những thứ văn chương thứ nhất của con người chúng ta vẫn không thể học thật đẹp. Lá cây thẳng trắng, nhưng trông cho một làn những chiếc lá nhỏ xinh đẹp trong lành và th

n vật. Không chỉ tài tình ở nghệ thuật miêu tả diễn biến tâm lí nhân vật. Tô Hoài còn tài tình khi mà có chiều người và cây gạo hơn, mặc áo dài của một người cho những cơn gió thổi lại thiêng liêng của các bạn trẻ quên được một cuộc đời để đánh giá của bà là chỗ chuyển thuộc vào một cái chuồng ngày xuống đất là những chiếc áo màu xanh của lòng chú hoa cúc trên cành là một cái cày, chắc chắn đã được sống trong cuộc sống. Chú đã được trang trí thanh niên trong một chiếc xe máy cho chiếc cặp sách của mình. Chúng là con gái của các bạn cùng nhau như cái gió thổi chưa hoặc trong lúc tròn và cánh đồng lúa chín chuối cho chiếc áo nhỏ của bố mẹ đã thấy bốn cái đuôi màu xanh nhỏ xíu, có màu xanh đậm như một người nhỏ màu cam của bố mẹ trồng cây cầu của chú chim ra trường trên các thầy cô. Chúng em thấy bố mẹ cho những chú gấu bông lúa chín cho cánh đồng lúa thật khỏe mạnh. Trên cánh đồng lúa trong vườn tranh thật nhưng lại có một cái hai bên bờ sông trong mặt trời xanh như vậy. Chúng tôi đã thấ

à về quan niệm nhà văn kiểu mới: nhà văn - chiến sĩ.
- Từ năm 1945 đến năm 1975, trên đất nước ta diễn ra trong tình yêu thương của thiên nhiên của con người. Chính trong tâm trạng nghệ thuật chính là những người bạn thân thiết của mỗi người. Ngày nay, mỗi người có ý nghĩa nhân văn là những chiếc lá nhỏ xíu như trong lúc nào cũng được người đã có bạn có một người thường phải chịu tai nạn giao thông. Để có thể thấy được cho chúng ta cần phải đi xe đạp của chúng ta vươn lên thường chuyển đến nhà trường và thiếu thi và điều đó sẽ có chuyện của mình.
Trong cuộc sống của mình là những người không chỉ là những điều đã được thương tiếc của người nhà trường là những trái tim yêu làng ngày càng trở nên thật là một cách sách vở để cho chúng ta thường phải nhận lỗi chính quyền làm người và phụ nữ là một thế giới cho con cháu nghe tiếng chiếc cặp trang trí đặc biệt là thành phố Hồ Chí Minh. Nhưng dẫn chứng có những thái độ và các thầy cô giáo thành công và vận dụng và đánh nhau và học giỏi của mìn

...


Iteration: 57, Loss: 1.092672

m giặc mà đánh. Việt chính là 1 hình tượng nhân vật điển hình cho tầng lớp thanh niên thời đánh Mỹ thì chúng ta không quên trái tim của mình. Trong thi cử thì chúng ta cần phải làm gì để giữ gìn giáo dục và cho mọi người nghĩ đến thành công của mình.
- Khi đi trên con đường trước mặt của nhà văn đã từng nói sử dụng ngôi chùa trong những ngày tháng gian ta trong thế giới đã bị thương hoành hành của ba mươi năm trên con đường này.
Các bạn trong tổ chị nghiêm khắc thì lại có những chiếc đồng hồ có ba mươi năm thứ ba, thật là vui sướng. Và tôi vẫn rất vui và đẹp. Những điều thú vị là những bạn các nhóm chúng ta những điều đầy thương thân của mình.
Trong các ngày mùa thu, ta cảm thấy thật là đẹp đẽ và đẹp đẽ và đẹp. Những thay đổi đến phần chín tháng ba mươi năm xa chiều rộng mênh mông. Một cái bút chì hồng thanh minh trên những chiếc mõm nhỏ xíu như thế nhưng chúng em cũng được chúng ta nhớ đến một khoảng thời gian để giữ gìn cây bút chì. Một hôm chú ta còn 

ới mấy đứa trẻ con trong xóm mỗi khi đi làm về. Xấu trai là thế, nghèo là thế nhưng anh Tràng lại nh⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒ỞỨỦỦỨ→ỨÊỞ*ỞỞỞỞZ→⇒Ở→—⇒ỨỨ→Z——⇒⇒⇒⇒Ứ⇒ỨỨ⇒⇒⇒⇒⇒⇒Ứ⇒⇒Ở⇒⇒⇒⇒Ở⇒⇒Ủ⇒ỦỨỨỨỨỞỨỨỨỨ⇒⇒ỨỨ⇒⇒⇒⇒⇒Ứ⇒⇒⇒ỨỨỞ⇒⇒⇒Ð⇒⇒⇒⇒Ứ—⇒⇒⇒⇒Ứ—Ứ—⇒ỨỨ⇒⇒⇒⇒⇒⇒→⇒→⇒ỨÂ⇒⇒⇒Ứ——⇒ỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨỨ⇒ỨỨỨỨỨỨỨỨỨỨỨỨỨ⇒⇒⇒⇒⇒⇒⇒ỨỨỨỨ⇒⇒⇒ỨỨỨỨ⇒Á⇒⇒ỨỨỨ⇒⇒Ứ⇒⇒ỨỨỨỨỨỨỨỨỨỨỨ⇒⇒ỨỨỨỨỨ⇒⇒ỨỨỨỨỨỨỨỨỨỨỨỨ⇒⇒ỨỨỨỨỨ⇒⇒⇒Ở⇒⇒ỨỨỨỨ⇒Ứ«⇒Ứ⇒Ứ⇒⇒⇒Ứ⇒⇒««Á⇒Ý—ỨỨỨỞỞỞỞỨỨỨỞỨ⇒Ứ⇒⇒ỨỨ⇒Ứ⇒⇒→→É→⇒⇒⇒⇒⇒⇒→⇒⇒⇒⇒⇒Ứ⇒⇒É⇒⇒→→→⇒⇒Ủ→⇒ỞỨ⇒⇒⇒⇒→—⇒⇒→—ỨÁ⇒⇒ỞỨ⇒⇒Á⇒⇒⇒Ứ⇒Ứ⇒⇒⇒Ứ⇒⇒ỨỨ⇒⇒⇒⇒ỨỨỞ⇒⇒⇒⇒⇒⇒ÉỞ⇒⇒⇒⇒⇒Ứ⇒ÉÐ—⇒⇒ỨỨ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỨỨỨỨ⇒⇒⇒⇒⇒⇒⇒ỨỨ⇒ỨỨỨỨỨỨ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỦỨỨ⇒⇒ÝỨ⇒→Ứ→→Ý⇒Ứ⇒⇒Ứ⇒⇒⇒⇒→⇒→→→→→ỦỞ⇒⇒⇒Ứ⇒Ứ⇒⇒⇒⇒⇒⇒⇒Ứ⇒—ẢỦ—Z⇒—⇒ÐỞỞ⇒⇒⇒⇒ÉỞỞỞỞỨỨỦỨỞỨỨ⇒⇒→ỞỨỞỨỞỞ⇒ỨỨ⇒⇒⇒Ở⇒⇒ÐỨ⇒⇒⇒⇒ỞỨ⇒⇒Ở⇒⇒⇒⇒⇒⇒Ứ⇒⇒⇒⇒ĂỨỐ→→→⇒⇒Ý⇒Ý⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỨỨ⇒Ứ⇒⇒Ố→ÐZ⇒ỞỞZZ———ÉỞ————⇒ỞỨ⇒ỨỨỨỨ⇒—⇒⇒⇒⇒Ứ⇒Ứ⇒⇒ÉỞỦỦ⇒Ứ⇒ÂỨỨỦỞỞ⇒Ủ⇒ỞỨ⇒⇒Ở⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỨỨỨ—⇒⇒Ứ⇒ỨỨ⇒⇒⇒⇒Ở⇒⇒⇒⇒⇒ỨÉỞ⇒→ỨÉ⇒Ứ⇒ỨỨ⇒ỞỞÐỨỨỨ⇒ỨỨỨỨỨỨỨỨ⇒⇒⇒⇒⇒⇒ỨỨỨỦỨỨỨỨỨỨ⇒Ứ⇒⇒⇒⇒⇒⇒⇒→⇒⇒⇒É⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ÉỨÉÂ⇒Ứ⇒Ứ⇒⇒⇒⇒⇒⇒⇒→Ứ⇒⇒Ứ⇒⇒Ứ→—Ứ⇒⇒Ứ⇒ỨỨ⇒⇒⇒⇒→⇒⇒→⇒⇒→Ứ⇒⇒⇒⇒É«Ứ→Ð⇒Ứ⇒⇒⇒Ở———⇒⇒⇒«———⇒Ở⇒Ý⇒⇒⇒⇒⇒ỨỨỨ⇒ỨỨỨỨ⇒ỨỨỨỨỨỨ⇒ỨỨỨ⇒ỨỨ⇒⇒⇒ÉÉ—⇒ÉZZỨỨỨ⇒ỞỞỞỨÉỞỞỨÉỞỞỨỨ⇒⇒⇒ÉỨ⇒⇒⇒Ứ⇒ỨÁZ⇒⇒⇒ỨỨ⇒⇒É⇒⇒⇒⇒⇒ỨỨỨ

yễn Tuân đã dâng trào mãnh liệt khi bắt gặp khung cảnh tuyệt đẹp đó. Một sự biến chuyển đầy tinh tế và độc lập tự do của Nguyễn Đình Chiểu. Hình ảnh đất nước là tình cảm của người phụ nữ trong truyền thuyết là bức tranh tuyệt vời của nước ta và đẹp. Các câu thơ trên của tác giả có nhiều giá trị văn học Việt Nam của Nguyễn Trãi, trong bài thơ “Sông Đà” của Nguyễn Trãi đã từng thấy hoàn cảnh khác nhau. Nhà thơ đã trở thành một điệp ngữ đang được tác giả đề cập đến bài thơ. Văn chương có nhiều tác phẩm thơ ca dân tộc được sử dụng theo ý kiến cho nhân dân ta và truyền thuyết đã trở thành một điểm chính trị và ước mơ. Và tác giả đã có một thành công của người dân chài thuộc địa giao thông được viết được một tác phẩm nổi tiếng của dân tộc.
- Truyện ngắn Chiều tự thưởng theo thể loại tác phẩm Truyền thống của dân tộc ta đều được phát triển và tinh thần thơ ca chân thành. Trong đoạn trích "Vô cảm" là một tác phẩm của Nguyễn Đình Chiểu cũng đã được thể hiện qua câu ca dao tục ngữ trên của truyệ

ng sản đề xướng và lãnh đạo, kinh tế nước ta từng bước chuyển sang nền kinh tế thị trường, văn hóa như vậy mà tuyệt đẹp của các bạn trong cuộc sống.
Lần thứ hai của ông thường là loại cây trong câu chuyện này thì văn hóa chính là nghi thức để đi làm trong công việc và xã hội chúng ta chúng ta trở thành tấm gương về tinh thần của mỗi người. Trong cuộc sống, con người trong cuộc sống của họ đã chuyển đến những thói xấu để đưa ra được các bạn học sinh để tham gia các hoạt động chúng ta phải chuyển sang chúng ta. Thông qua câu chuyện thường có thể thấy ông lão nhà trường còn có thể là người thầy và điều đó của mình trong học tập. Tôi cảm thấy không biết chú đã học để cho chúng tôi một người thầy cô là người của ông lão đánh thắng cho mình một chút nhưng có thể nói là chú đã vào lòng tôi cũng như con để báo đáp công ơn người thân. Những trưa hè nóng bức, người thầy cô còn có thể nhận ra được những điều tốt đẹp hơn.
Những lúc rảnh rỗi, em lại thấy chú gấu ra đồng để có thể chuyển đến nhà chơ

KeyboardInterrupt: 

In [351]:
def sample_network(istrain=False):
    if istrain == True:
        train()
    else:
        parameters = pickle_compress('params.pickle') # load params
        generated = generate_output(parameters, get_input=True)
        print(generated)
sample_network(False)

Nhập câu đầu: cháu lên ba
abc


C:\Users\user\anaconda3\envs\ve\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


cháu lên ba⇒ỞỞ⇒⇒ÂỨỞỞ⇒⇒⇒Ở⇒⇒⇒⇒⇒⇒⇒Ở⇒ỞỦ⇒ỨỨỨ⇒⇒⇒→→Ứ⇒Ứ→⇒⇒⇒⇒⇒⇒Ở⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Z⇒⇒⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒Ở⇒Á⇒Ứ⇒⇒⇒⇒→⇒⇒⇒⇒⇒⇒ỞỞ⇒ỨỞỞỨ⇒Ở⇒ỨỞÉÉỨ⇒⇒ỨỨỨ⇒⇒⇒⇒⇒⇒→«ỨÐỨ⇒→É⇒→Ứ⇒ỨZỨ⇒Ứ⇒⇒⇒⇒⇒⇒⇒ỨỞỨỨỨỨỨỨỨỨỨ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỞỨỨỨỨỨ⇒Ứ—ỨỞ⇒→ỨỨỨỨỨỨỨ⇒⇒⇒Ứ—Z⇒⇒⇒⇒Ứ⇒⇒⇒⇒Ứ⇒⇒⇒⇒⇒ỨỨỨỨỨỨ⇒ZỞZZ⇒⇒ỨỨỨỨỨỨỨỨỨỨỨỨỨZỨỨ⇒Ứ⇒Ứ⇒Ứ⇒Ứ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ÝÝÉÉÉ⇒Ứ⇒Ý⇒Ý⇒⇒⇒Ứ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒⇒Ứ⇒⇒⇒Ý⇒⇒⇒Â⇒Z⇒⇒Ý⇒Ứ⇒ỨỨỨỨỨỨỨ⇒ỨỨỨ⇒—Z⇒ỨỨ⇒ZZ———⇒Z—⇒⇒—Z————Ứ————ỞỞỨ⇒Ứ—Ứ⇒ỞỞỨỞỨỨỨỨỨỨỨỨ—Ứ⇒⇒Ở⇒Ứ⇒⇒ỞỨ⇒ỨÉỨ⇒ỞỨỨỨỨỨỨỨỨỨỨỨỨÉZ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ÉÐÉ⇒⇒⇒⇒⇒⇒⇒→⇒Ð⇒⇒⇒⇒ÐZ⇒ZÐZZÉÉZ→→⇒⇒ÂỞỞ⇒⇒ỞỞZ—Z⇒ỨÁỞÉ⇒⇒Ố⇒ÁÁ⇒⇒⇒⇒⇒⇒⇒Ứ⇒⇒⇒⇒⇒Ă⇒→⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ứ—⇒⇒Ở⇒⇒ỨZZ→⇒ỨZZỞỨÁÁ⇒ZZỞZZZ⇒ỞỞ⇒→—⇒⇒⇒⇒⇒⇒Ở⇒⇒⇒Ở⇒⇒ỞỞ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒→⇒⇒Ở⇒⇒⇒⇒⇒Ð⇒ỞỨ⇒⇒⇒⇒⇒⇒⇒⇒ỨỨỨỨÂỨ⇒Ở⇒ỦỨ⇒Ứ⇒Ứ⇒⇒ỞỞỞ⇒Ứ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒ỨỞỨỞỨỨỨ⇒⇒Ở⇒Ứ⇒⇒ỞỞỨ⇒⇒⇒⇒—⇒⇒⇒⇒⇒→⇒⇒⇒⇒→→⇒⇒⇒⇒→⇒⇒⇒⇒⇒⇒ỞỞ⇒⇒ỞỞ⇒Ố⇒Ở⇒⇒Ứ⇒ZỞ⇒ỞỞ⇒Ở⇒Ứ⇒⇒⇒Ở⇒⇒⇒⇒Ở⇒Ở⇒ZZ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒É⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒—⇒⇒⇒Z⇒ỞỞ⇒Ứ⇒Ở⇒Ở⇒⇒Ở⇒⇒⇒Ứ⇒⇒⇒⇒⇒⇒⇒Ở⇒⇒É⇒⇒⇒⇒⇒Ứ⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Z⇒⇒⇒⇒⇒——⇒⇒⇒ỞỞỞ⇒⇒ỞỞ⇒Ở⇒⇒ỞỞ⇒⇒⇒ỞỨ⇒⇒⇒—Ð⇒⇒⇒ỨỨỨỞỞ⇒Ứ⇒⇒Ở⇒⇒⇒⇒⇒⇒⇒⇒ỞỞỞỞỞỞỞỞỞ⇒ỞỞ⇒Ứ⇒ỞỞ⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒Ý⇒Ở⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒⇒—⇒⇒⇒⇒Z⇒⇒É⇒⇒⇒⇒⇒⇒⇒⇒ÐỨ⇒ỨZ